Install modules

In [3]:
%pip install -U "ray[client]==2.9.0"
%pip install -U "ray[serve]==2.9.0"
%pip install transformers fastapi starlette

%pip install -U ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
import asyncio
import logging
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from ray import serve

# Set up logging
logger = logging.getLogger("ray.serve")

# Initialize FastAPI app
app = FastAPI()

@serve.deployment
@serve.ingress(app)
class Chatbot:
    def __init__(self, model_id: str):
        self.loop = asyncio.get_running_loop()
        self.model_id = model_id
        self.model = AutoModelForCausalLM.from_pretrained(self.model_id)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id)

    @app.websocket("/chat")
    async def chat(self, websocket: WebSocket):
        await websocket.accept()
        try:
            while True:
                data = await websocket.receive_text()
                logger.info(f"Received message: {data}")
                response = await self.generate_response(data)
                await websocket.send_text(response)
        except WebSocketDisconnect:
            logger.info("Client disconnected")

    async def generate_response(self, prompt: str) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt")
        streamer = TextIteratorStreamer(self.tokenizer)

        # Generate response in a background thread
        await self.loop.run_in_executor(None, self.model.generate, **inputs, streamer=streamer)

        # Collect generated tokens
        response = ""
        for token in streamer:
            response += token

        return response

# Bind and run the deployment
chatbot_app = Chatbot.bind("microsoft/DialoGPT-small")  # Replace with your model ID
serve.run(chatbot_app)

2024-11-13 08:56:12,452	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 
(ProxyActor pid=778018) INFO 2024-11-13 08:56:19,009 proxy 192.168.100.251 proxy.py:1128 - Proxy actor de8796fd9a7224e02b3c86e401000000 starting on node e51ca75891f8578bf0b66770221369bfe033829266d37e744c593f43.
(ProxyActor pid=778018) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-e51ca75891f8578bf0b66770221369bfe033829266d37e744c593f43:ProxyActor.__init__() (pid=778018, ip=192.168.100.251, actor_id=de8796fd9a7224e02b3c86e401000000, repr=<ray.serve._private.proxy.ProxyActor object at 0x7c360014aee0>)
(ProxyActor pid=778018)   File "/usr/lib/python3.8/concurrent/futures/_base.py", line 437, in result
(ProxyActor pid=778018)     return self.__get_result()
(ProxyActor pid=778018)   File "/usr/lib/python3.8/concurrent/futures/_base.py", line 389, in __get_result
(ProxyActor

RayActorError: The actor died unexpectedly before finishing this task.
	class_name: ProxyActor
	actor_id: de8796fd9a7224e02b3c86e401000000
	pid: 778018
	name: SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-e51ca75891f8578bf0b66770221369bfe033829266d37e744c593f43
	namespace: serve
	ip: 192.168.100.251
The actor is dead because it was killed by `ray.kill`.

(ProxyActor pid=778146) INFO 2024-11-13 08:56:24,629 proxy 192.168.100.251 proxy.py:1128 - Proxy actor 8bcd42f7b11b6ef7cf2fbb5d01000000 starting on node e51ca75891f8578bf0b66770221369bfe033829266d37e744c593f43.
(ProxyActor pid=778146) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-e51ca75891f8578bf0b66770221369bfe033829266d37e744c593f43:ProxyActor.__init__() (pid=778146, ip=192.168.100.251, actor_id=8bcd42f7b11b6ef7cf2fbb5d01000000, repr=<ray.serve._private.proxy.ProxyActor object at 0x7f0cf2109f10>)
(ProxyActor pid=778146)   File "/usr/lib/python3.8/concurrent/futures/_base.py", line 444, in result
(ProxyActor pid=778146)     return self.__get_result()
(ProxyActor pid=778146)   File "/usr/lib/python3.8/concurrent/futures/_base.py", line 389, in __get_result
(ProxyActor pid=778146)     raise self._exception
(ProxyActor pid=778146)   File "/home/danielpepuho/Desktop/Github/pra-ta/.venv/lib/